In [368]:
y_wave = pow(h, x2, p)
y_wave

79579958052646501458862762659181463507327558671984305947744525682632323264487747904473738543573763051771022054923877179971739948354892965023603986036785828872085838591102145343037263803231387634813975027113016288086841693808581613143788506046363292832869586430022949945483950735653589241022258421459693427909

In [369]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.backends import default_backend
import random
from hashlib import sha256, sha3_256


def generate_dsa_parameters():
    # Generate DSA parameters with 1024-bit p and 160-bit q
    parameters = dsa.generate_parameters(key_size=1024, backend=default_backend())
    
    # Generate private and public keys based on the generated parameters
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()

    # Access the DSA parameters
    p = parameters.parameter_numbers().p
    q = parameters.parameter_numbers().q
    g = parameters.parameter_numbers().g

    print("Generated DSA Parameters:")
    print("p:", p)
    print("q:", q)
    print("g:", g)
    print("\nPrivate Key:", private_key)
    print("Public Key:", public_key)
    return p,q,g

p,q,g = generate_dsa_parameters()

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
h = pow(g, h, p)
y_wave = pow(h, x2, p)

r, R = generate_keys(g, q)
e3 = int(sha256(str(str(msg) + str(Y) + str(pow(g, r, p)) + str(y_wave) + str(pow(h, r, p))).encode()).hexdigest(), 16) % q
print('e3:', e3)

_, random_s3 = generate_keys(g, q)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
calculation_1 = (pow(h, random_s3, p)*pow(y_wave, e3, p)) % p
e1 = int(sha256(str(str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
print('e1:', e1)

_, random_s1 = generate_keys(g, q)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
calculation_1 = (pow(h, random_s1, p)*pow(y_wave, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
print('e2:', e2)

s2 = (r - x2 * e2) % ((p-1)//2)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

Generated DSA Parameters:
p: 96074658718289819654852732191520997842367736533201940153401085738430648656296649220988031328783142299601018656019251294898939944025803973219772881430620274182160801634850526500682250886119138044367469605904036148799623924093773171726447933586407243141661355537358425879793611678878966389125679726993710591731
q: 1460273295577695843011364613799654976197879837537
g: 60234245278512950505569432712337640179765972309960456173442288205964448463810425662638659681644493751909990294667061396156186255053626339757596379407463891082127586423366227507039197473911025805952418609164525465457870430717527749477322667001734642080502751450106118306389913434182454366539624369705472008588

Private Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPrivateKey object at 0x00000276C540B710>
Public Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPublicKey object at 0x00000276C5409010>
msg: Hello, Ring Signature
pubkeys: [775040175107379010334624763827471522173324310249103

{'e1': 1257422836200334152636829932501079518027494066415,
 's1': 748696063812714438950849690144117409197557690781,
 's2': 20432120833742648517628831835845325467898450400773944970498901351674417067937153428689924842945819416995333087817773677686548736707483020783857807532989881853290161021086280484179732089978193110729461005266612541665526416369604485141367479292488039916471269309764996134382671248982773245790165075740730044167,
 's3': 1117490054659773885204427332906409412361305819261}

In [370]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
calculation_1 = (pow(h, total_res['s1'], p)*pow(y_wave, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
calculation_1 = (pow(h, total_res['s2'], p)*pow(y_wave, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
calculation_1 = (pow(h, total_res['s3'], p)*pow(y_wave, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)

1257422836200334152636829932501079518027494066415
749425345401069672404451462785610307040334761536
772442077564776133621565032723162571295968943470
True
